In [1]:
import numpy as np
from rembg import remove 
import io
from PIL import Image, ImageOps, ImageDraw
import cv2
from matplotlib import pyplot as plt

In [73]:
# Store path of the image in the variable input_path 
input_path =  'IMG_5657.JPEG' 
  
# Processing the image 
input = Image.open(input_path) 
  
# Removing the background from the given Image 
output = remove(input) 

alpha_matte = remove(input, alpha_matting=True, only_mask=True)

# Convert to NumPy array for processing
alpha_matte_np = np.array(alpha_matte)
  
kernelSizes = [(3, 3), (5, 5), (7, 7), (15, 15)]
# loop over the kernels sizes again
for kernelSize in kernelSizes:
	# construct a rectangular kernel form the current size, but this
	# time apply a "closing" operation
	kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernelSize)
	closing = cv2.morphologyEx(alpha_matte_np, cv2.MORPH_CLOSE, kernel)

# Convert back to PIL Image
refined_mask_image = Image.fromarray(closing)

# Apply refined mask to input image
output_image = Image.composite(input, Image.new("RGBA", input.size), refined_mask_image)

# # Store path of the output image in the variable output_path 
# output_path = 'E:\C programs\ 
#                Remove BackGround\gfgoutput.png' 

# #Saving the image in the given path 
# output.save(output_path) 

In [3]:
from backgroundremover.bg import remove
def remove_bg(src_img_path, out_img_path):
    model_choices = ["u2net", "u2net_human_seg", "u2netp"]
    f = open(src_img_path, "rb")
    data = f.read()
    img = remove(data, model_name=model_choices[0],
                 alpha_matting=False,
                #  alpha_matting=True,
                #  alpha_matting_foreground_threshold=240,
                #  alpha_matting_background_threshold=10,
                #  alpha_matting_erode_structure_size=10,
                #  alpha_matting_base_size=1000
                )
    f.close()
    f = open(out_img_path, "wb")
    f.write(img)
    f.close()

In [ ]:
remove_bg('IMG_5657.JPEG', 'IMG_5657_3.png')

In [2]:
def add_white_stroke(image_path, stroke_width):
    # Open the image and convert to RGBA
    img = Image.open(image_path).convert("RGBA")
    
    # Convert image to numpy array
    img_np = np.array(img)
    
    # Extract the alpha channel
    alpha = img_np[:, :, 3]
    
    # Create a binary mask where the object is white and the background is black
    mask = cv2.threshold(alpha, 0, 255, cv2.THRESH_BINARY)[1]
    
    # Find contours (edges) of the object
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create an empty image for the stroke
    stroke_img = Image.new("RGBA", img.size, (255, 255, 255, 0))
    stroke_draw = ImageDraw.Draw(stroke_img)
    
    # Draw the contours with a white stroke
    for contour in contours:
        # Offset the contour points by half the stroke width to center the stroke
        offset_contour = [(point[0][0] + stroke_width // 2, point[0][1] + stroke_width // 2) for point in contour]
        # Draw the stroke by expanding the contour
        stroke_draw.line(offset_contour, fill=(255, 255, 255, 255), width=stroke_width)
    
    # Composite the original image with the stroke
    combined = Image.alpha_composite(stroke_img, img)
    
    return combined

# Usage
image_path = 'img.png'
stroke_width = 50  # Stroke width in pixels

stroked_image = add_white_stroke(image_path, stroke_width)
stroked_image.show()  # Display the image with the stroke
# stroked_image.save('stroked_object_image.png')  # Save the image with the stroke


In [3]:
image_path = 'IMG_5657_3.png'

# Open the image and convert to RGBA
img = Image.open(image_path).convert("RGBA")

# Convert image to numpy array
img_np = np.array(img)

# Extract the alpha channel
alpha = img_np[:, :, 3]

# Create a binary mask where the object is white and the background is black
mask = cv2.threshold(alpha, 0, 255, cv2.THRESH_BINARY)[1]

# Find contours (edges) of the object
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [4]:
# Create an empty image for the stroke
stroke_img = Image.new("RGBA", img.size, (255, 255, 255, 0))
stroke_draw = ImageDraw.Draw(stroke_img)

stroke_width=50

# Draw the contours with a white stroke
for contour in contours:
    # Offset the contour points by half the stroke width to center the stroke
    offset_contour = [(point[0][0] + stroke_width // 2, point[0][1] + stroke_width // 2) for point in contour]
    # Draw the stroke by expanding the contour
    stroke_draw.line(offset_contour, fill=(255, 255, 255, 255), width=stroke_width)

# Composite the original image with the stroke
combined = Image.alpha_composite(stroke_img, img)

In [5]:
combined.show()

In [6]:
# Let's load a simple image with 3 black squares 
image = cv2.imread('IMG_5657_3.png') 
# cv2.waitKey(0) 
  
# Grayscale 
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
# Find Canny edges 
edged = cv2.Canny(gray, 30, 200) 
cv2.waitKey(0) 
  
# Finding Contours 
# Use a copy of the image e.g. edged.copy() 
# since findContours alters the image 
contours, hierarchy = cv2.findContours(edged,  
    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
  
cv2.imshow('Canny Edges After Contouring', edged) 
cv2.waitKey(0) 
  
print("Number of Contours found = " + str(len(contours))) 
  
# Draw all contours 
# -1 signifies drawing all contours 
cv2.drawContours(image, contours, -1, (0, 255, 0), 3) 
  
cv2.imshow('Contours', image) 
cv2.waitKey(0) 
cv2.destroyAllWindows() 

Number of Contours found = 281
